# Objetivo

En este trabajo vamos a tratar de implementar el modelo de evaluación de riesgo de la epidemia desarrollado por el equipo de la [Computational Biology and Complex Systems; Universitat Politècnica de Catalunya - BarcelonaTech](https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf) . Para obtener los datos epidemiológicos nos apoyamos en el grupo de trabajo [#escovid19data](https://github.com/montera34/escovid19data).


@author: Manuel H. Arias 

@Twitter: @walyt

@mail: mharias@me.com



## Importación de librerias estándares para DAE

In [1]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib import cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.dates import (YEARLY, MONTHLY, DateFormatter, MonthLocator,DayLocator,
                              rrulewrapper, RRuleLocator, drange)
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import matplotlib.colors as colors

import numpy as np
from datetime import datetime,timedelta
import seaborn as sns
%matplotlib inline

import urllib.request

import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

## Descargamos la información

Hacemos en primer lugar una actualización de parámetros y preparación de variables que necesitaremos durante el ejercicio

In [2]:
pd.options.display.max_rows = 999 #Variable de contexto para permitir la presentación de datos por pantalla
pd.set_option('display.max_columns', None)

In [3]:
#url de este notebook en github
github_url = 'https://github.com/mharias/covid/blob/master/graficos_escovid19data.ipynb'

In [4]:
hashtag_fuente='#escovid19data' #hashtag de la fuente para los créditos

In [5]:
#url de la fuente de datos
path_montera34='https://github.com/montera34/escovid19data/blob/master/data/output/covid19-provincias-spain_consolidated.csv?raw=true'

Leemos los datos en un `pandas`

In [6]:
df = pd.read_csv(path_montera34)

veamos una rápida descripción de la información:

In [7]:
df.describe()

,ine_code,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7,ia14
count,13844.000000,4355.000000,3925.000000,959.000000,1560.000000,9906.000000,9769.000000,11038.000000,7080.000000,8312.000000,7146.000000,13822.000000,13822.000000,13822.000000,13822.000000,13822.000000,1.384400e+04,7080.000000,9769.000000,11038.000000,9906.000000,6228.000000,6607.000000,7602.000000,7922.000000,6902.000000,6538.000000,8888.000000,8524.000000,10970.000000,10276.000,10270.000000,10674.000000,9223.000000,13510.000000,8318.000000
mean,26.500506,59.710677,107.141146,7394.139729,1116.169872,179.139209,24.562391,580.072477,5574.639124,8639.229548,1817.836672,75.065765,69.822891,1.496455,2.384749,1.361670,9.027265e+05,504.401037,2.661817,53.204786,20.362552,1202.597784,597.880884,1430.723494,720.813683,85.335990,84.505659,101.150428,98.381664,3.832543,inf,3.901675,3.782631,28.782934,70.330155,129.806805
std,15.012545,131.600451,217.756238,21865.327207,1575.616462,778.034928,89.359107,1784.792894,13670.555825,24407.737153,2646.839839,265.284175,255.028671,6.512300,35.657480,9.714829,1.173114e+06,542.447865,3.850305,51.047434,41.797412,2644.819562,1363.871568,4309.897169,2170.062155,215.895721,194.546805,332.692405,299.737414,20.220796,NaN,20.493709,19.907835,149.043201,244.037309,159.976616
min,1.000000,-67.000000,-67.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.477700e+04,0.000000,0.000000,0.000000,0.000000,-54.000000,-82.000000,-62.000000,-99.000000,-106.000000,-12.000000,-84.000000,-12.700000,-22.000000,-30.800,-1.300000,-3.300000,-4.000000,0.000000,-7.000000
25%,13.000000,3.000000,6.000000,3.000000,172.750000,11.000000,2.000000,91.000000,722.000000,1470.500000,429.000000,2.000000,1.000000,0.000000,0.000000,0.000000,3.167980e+05,128.415000,0.300000,13.130000,2.410000,125.000000,55.000000,46.000000,21.000000,5.000000,8.000000,2.000000,3.100000,0.000000,0.000,0.000000,0.000000,1.000000,1.700000,9.000000
50%,26.500000,22.000000,37.000000,23.000000,520.000000,44.000000,6.000000,206.000000,2049.500000,3002.000000,1200.500000,14.000000,11.000000,0.000000,0.000000,0.000000,5.810780e+05,325.025000,1.300000,34.285000,8.690000,516.000000,239.000000,279.000000,137.000000,28.000000,33.900000,17.000000,19.100000,0.000000,0.000,0.700000,0.600000,4.000000,11.900000,58.000000
75%,40.000000,63.000000,112.000000,482.500000,1306.000000,127.000000,20.000000,373.000000,4603.000000,5776.250000,2141.000000,65.000000,59.000000,0.000000,0.000000,0.000000,1.022800e+06,707.425000,3.470000,83.100000,23.037500,1125.000000,558.000000,1147.750000,589.000000,79.000000,79.375000,80.000000,80.300000,2.000000,0.925,2.300000,2.100000,16.000000,61.300000,205.000000
max,52.000000,1992.000000,3444.000000,117877.000000,10275.000000,15227.000000,1528.000000,11888.000000,149640.000000,296929.000000,29753.000000,6743.000000,6730.000000,128.000000,1185.000000,261.000000,6.663394e+06,3687.080000,28.720000,245.080000,474.750000,29954.000000,17440.000000,52884.000000,28390.000000,3450.000000,2491.400000,6832.000000,4055.700000,377.000000,inf,350.700000,339.000000,2373.000000,4168.400000,1005.000000


y un muestreo de valores y de algunas columnas de interés:

In [8]:
df.head()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
0,2020-01-31,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
1,2020-02-01,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
2,2020-02-02,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
3,2020-02-03,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN
4,2020-02-04,Almería,4,Andalucía,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,716820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ISCIII RENAVE,https://cnecovid.isciii.es/covid19/resources/d...,NaN


In [9]:
df.tail()

,date,province,ine_code,ccaa,new_cases,PCR,TestAc,activos,hospitalized,intensive_care,deceased,cases_accumulated,cases_accumulated_PCR,recovered,num_casos,num_casos_prueba_pcr,num_casos_prueba_test_ac,num_casos_prueba_otras,num_casos_prueba_desconocida,poblacion,cases_per_cienmil,intensive_care_per_1000000,deceassed_per_100000,hospitalized_per_100000,cases_14days,cases_7days,cases_PCR_14days,cases_PCR_7days,daily_cases,daily_cases_avg7,daily_cases_PCR,daily_cases_PCR_avg7,daily_deaths,daily_deaths_inc,daily_deaths_avg3,daily_deaths_avg7,deaths_last_week,num_casos_prueba_pcr_avg7,ia14,source_name,source,comments
13839,2020-10-18,"Rioja, La",26,"Rioja, La",NaN,94.0,NaN,NaN,99.0,23.0,439.0,NaN,10591.0,NaN,108.0,82.0,0.0,26.0,0.0,316798,NaN,7.26,138.57,31.25,NaN,NaN,NaN,NaN,NaN,NaN,94.0,136.2,3.0,NaN,NaN,1.5,NaN,123.3,NaN,Ministerio de Sanidad;ISCIII RENAVE,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
13840,2020-10-19,"Rioja, La",26,"Rioja, La",NaN,137.0,NaN,NaN,100.0,20.0,439.0,NaN,10722.0,NaN,89.0,73.0,0.0,14.0,2.0,316798,NaN,6.31,138.57,31.57,NaN,NaN,NaN,1217.0,NaN,NaN,131.0,136.1,0.0,0.0,NaN,1.4,NaN,115.1,NaN,Ministerio de Sanidad;ISCIII RENAVE,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
13841,2020-10-20,"Rioja, La",26,"Rioja, La",NaN,179.0,NaN,NaN,108.0,21.0,440.0,NaN,10876.0,NaN,86.0,80.0,0.0,6.0,0.0,316798,NaN,6.63,138.89,34.09,NaN,NaN,NaN,1207.0,NaN,NaN,154.0,134.7,1.0,0.2,1.3,1.4,NaN,106.6,NaN,Ministerio de Sanidad;ISCIII RENAVE,https://www.mscbs.gob.es/profesionales/saludPu...,NaN
13842,2020-10-21,"Rioja, La",26,"Rioja, La",NaN,238.0,NaN,NaN,111.0,22.0,441.0,NaN,11121.0,NaN,90.0,84.0,0.0,6.0,0.0,316798,NaN,6.94,139.21,35.04,NaN,NaN,2011.0,1270.0,NaN,NaN,245.0,143.7,1.0,0.2,0.7,1.4,NaN,100.3,635.0,Ministerio de Sanidad;ISCIII RENAVE,https://www.mscbs.gob.es/profesionales/saludPu...,El número acumulado de casos y de fallecidos c...
13843,2020-10-22,"Rioja, La",26,"Rioja, La",NaN,174.0,NaN,NaN,116.0,22.0,442.0,NaN,11300.0,NaN,3.0,3.0,0.0,0.0,0.0,316798,NaN,6.94,139.52,36.62,NaN,NaN,2079.0,1319.0,NaN,NaN,179.0,150.7,1.0,0.2,1.0,0.8,NaN,81.7,656.0,Ministerio de Sanidad;ISCIII RENAVE,https://www.mscbs.gob.es/profesionales/saludPu...,El número acumulado de casos y de fallecidos c...


In [10]:
df['province'].unique()

array(['Almería', 'Cádiz', 'Córdoba', 'Granada', 'Huelva', 'Jaén',
       'Málaga', 'Sevilla', 'Huesca', 'Teruel', 'Zaragoza', 'Asturias',
       'Balears, Illes', 'Palmas, Las', 'Santa Cruz de Tenerife',
       'Cantabria', 'Albacete', 'Ciudad Real', 'Cuenca', 'Guadalajara',
       'Toledo', 'Ávila', 'Burgos', 'León', 'Palencia', 'Salamanca',
       'Segovia', 'Soria', 'Valladolid', 'Zamora', 'Barcelona', 'Girona',
       'Lleida', 'Tarragona', 'Ceuta', 'Alicante/Alacant',
       'Castellón/Castelló', 'Valencia/València', 'Badajoz', 'Cáceres',
       'Coruña, A', 'Lugo', 'Ourense', 'Pontevedra', 'Madrid', 'Melilla',
       'Murcia', 'Navarra', 'Araba/Álava', 'Bizkaia', 'Gipuzkoa',
       'Rioja, La'], dtype=object)

Vamos a formatear correctamente la columna `Date`, para posteriormente ordenar el $pandas$ por fecha creciente:

In [11]:
df['date']=pd.to_datetime(df['date'])

In [12]:
df.columns

Index(['date', 'province', 'ine_code', 'ccaa', 'new_cases', 'PCR', 'TestAc',
       'activos', 'hospitalized', 'intensive_care', 'deceased',
       'cases_accumulated', 'cases_accumulated_PCR', 'recovered', 'num_casos',
       'num_casos_prueba_pcr', 'num_casos_prueba_test_ac',
       'num_casos_prueba_otras', 'num_casos_prueba_desconocida', 'poblacion',
       'cases_per_cienmil', 'intensive_care_per_1000000',
       'deceassed_per_100000', 'hospitalized_per_100000', 'cases_14days',
       'cases_7days', 'cases_PCR_14days', 'cases_PCR_7days', 'daily_cases',
       'daily_cases_avg7', 'daily_cases_PCR', 'daily_cases_PCR_avg7',
       'daily_deaths', 'daily_deaths_inc', 'daily_deaths_avg3',
       'daily_deaths_avg7', 'deaths_last_week', 'num_casos_prueba_pcr_avg7',
       'ia14', 'source_name', 'source', 'comments'],
      dtype='object')

# Gráfico de BiocomSC

## preparamos el Pandas

In [13]:
# Gráfico de BiocomSC


provincias=['Madrid']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_PCR_14]\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),'daily_cases_PCR'].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),'daily_cases_PCR'].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),'daily_cases_PCR'].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),'daily_cases_PCR'].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

In [14]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
#y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
#x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    #print (i)
    #ax.set_xlim(0,x1)
    #ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Madrid\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)


lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [15]:
len(datos.index)

234

In [16]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

In [17]:
anim.save('modelo_biocomsc_madrid.mp4',dpi=288)

# Gráfico de BiocomSC Badajoz

In [18]:
provincias=['Badajoz']
datos = df.fillna(0,axis=1)
datos = datos[(datos['date']>=datetime(year=2020,month=3,day=1)) & (datos['province'].isin(provincias))]

clave_avg='daily_cases_PCR_avg7'
clave_ratio_avg = 'ratio_daily_cases_PCR_avg7'
color_avg = 'darkred'
color_titulos = 'navy'
color_diarios = 'royalblue'
clave_ratio = 'tasa_ia_7_dias'
AI='incidencia_acumulada_14'
casos_PCR_14='cases_PCR_14days'
casos_nuevos = 'daily_cases_PCR'
casos_nuevos = 'new_cases'
clave_casos_popu = 'casos_per_population'
ventana_IA=14
titulo = 'Incidencia Acumulada de 14 días de PCR+ por 100.000 hab \n Ratio de la IA día(0)/día(-7)'
texto_y1='IA a 14 días'
texto_y2='Ratio a 7 días'

un_dia=timedelta(days=1)
dos_dias=timedelta(days=2)
cinco_dias=timedelta(days=5)
seis_dias=timedelta(days=6)
siete_dias=timedelta(days=7)
  
for provincia in provincias:
    
    datos.loc[datos['province']==provincia,AI]=\
    datos.loc[datos['province']==provincia,casos_nuevos].rolling(ventana_IA).sum()\
    /datos.loc[datos['province']==provincia,'poblacion']*100000

    datos.loc[datos['province']==provincia,clave_ratio] = \
    datos.loc[datos['province']==provincia][AI].pct_change(periods=7).add(1)    
    
    datos.loc[datos['province']==provincia,clave_casos_popu]=\
    datos.loc[datos['province']==provincia,casos_nuevos]\
    /datos.loc[datos['province']==provincia,'poblacion']*1000000

#datos = datos.set_index('date')[[casos_PCR_14,clave_ratio]]   
  
def calculo_p7(x):
    fecha_actual=x['date']
    #print (fecha_actual)
    if fecha_actual>datetime(year=2020,month=3,day=8):
        valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-un_dia),casos_nuevos].values[0]+\
        datos.loc[datos['date']==(fecha_actual-dos_dias),casos_nuevos].values[0])/\
        (datos.loc[datos['date']==(fecha_actual-cinco_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-seis_dias),casos_nuevos].values[0]+\
         datos.loc[datos['date']==(fecha_actual-siete_dias),casos_nuevos].values[0])
    else:
        valor = 0
    return valor


datos['p7']=datos.apply(calculo_p7,axis=1)
datos['p7_avg7'] = datos['p7'].rolling(window=7).mean()
datos = datos.reset_index()
datos = datos.drop(['index'],axis=1)

<ipython-input-18-3226f7d8de1c>:46: RuntimeWarning: divide by zero encountered in double_scalars
  valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\
<ipython-input-18-3226f7d8de1c>:46: RuntimeWarning: invalid value encountered in double_scalars
  valor = (datos.loc[datos['date']==fecha_actual,casos_nuevos].values[0]+\


In [19]:
color_rojo = sns.color_palette("bright", 10)[3]
color_verde = sns.color_palette("muted", 10)[2]
color_amarillo = sns.color_palette("YlOrRd", 10)[1]
eje_X = ' Incidencia Acumulada a 14 días'
eje_Y = 'Parámetro \u03C1'+'7'

fuente_modelo = 'https://biocomsc.upc.edu/en/shared/20200412_report_web_27.pdf'


y0, y1 = (-0.21951443768357135, 4.609803191354998)
y= 0, 2

x0, x1 = (-26.666590629339943, 817.5248229355791)
x0, x1 = 0,300
rango = np.arange(x0,x1,1)
def formateamos_eje(ax):
    '''
    función que formatea el gráfico con las características que queramos
    '''
    ax.set_facecolor('.8')
    ax.tick_params(labelsize=8, length=0)
    ax.grid(True, axis='x', color='white')
    ax.set_axisbelow(True)
    [spine.set_visible(False) for spine in ax.spines.values()]

    

def init():
    '''
    función que inicia el eje en el que dibujamos las barras
    
    '''
    ax.clear()
    formateamos_eje(ax)
    #ax.set_ylim(.2, 6.8)



def update(i):
    '''
    Función invocada para ir regenerando el gráfico, 
    Va actualizando añadiendo días (i incrementandose).
    
    '''
    for elemento in lista_elementos:
        elemento.remove()
    lista_elementos[:] = []
    
    
    
    #print (i)
    ax.set_xlim(0,x1)
    ax.set_ylim(0,y1)
    #colores = sns.color_palette("YlOrBr",i)
    colores = sns.color_palette("Blues",i)
    ax.scatter(datos[AI][:i],datos['p7_avg7'][:i],color=colores[:i:],marker='o',
               s=5)
    #ax.xaxis.set_major_locator(DayLocator([1]))
    #majorFmt = DateFormatter('%-d-%b')
    #ax.xaxis.set_major_formatter(majorFmt)
    ax.grid(True,axis='both')
    ax.yaxis.set_tick_params(labelsize=0,labelcolor=color_avg,width=0)
    ax.set_ylabel(eje_Y,size=14,color='black')
    ax.set_xlabel(eje_X,size=14,color='black')
    #ax4.set_yticks([])
    ax.yaxis.set_tick_params(labelsize=16,labelcolor=color_avg,width=0)
        
    ax.yaxis.set_tick_params(labelsize=12)
    ax.xaxis.set_tick_params(labelsize=12)
    
    for indice in range(i+1):
        if indice % 15 == 0 or indice==7 or indice == (len(datos.index)-1):
            try:
                color_texto = colores[indice-1]
            except:
                color_texto = 'black'
            style = dict(size=8, color=color_texto,weight='bold')
            texto = ax.annotate(datos.iloc[indice]['date'].strftime('%B %-d'),
                                xy=(datos.iloc[indice][AI],datos.iloc[indice]['p7_avg7']),
                                xycoords='data',
                                xytext=(-10, 10), 
                                textcoords='offset points',
                                **style)
            lista_elementos.append(texto)
    
    
    #date_str = datos.index[i].strftime('%B %-d, %Y')
    ax.hlines(1,x0,x1,colors='black',linestyles='dotted')
    
    ax.fill_between(rango, 0, 30/rango,
                 facecolor="blue", # The fill color
                 color=color_verde,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango, 30/rango, 100/rango,
                 facecolor='blue', # The fill color
                 color=color_amarillo,       # The outline color
                 alpha=0.02) 
    ax.fill_between(rango,100/rango,10,
                 facecolor="blue", # The fill color
                 color=color_rojo,       # The outline color
                 alpha=0.02)
    
    
    ax.set_title(f' Evolución de Empiric Potential Growth en Badajoz\n Fuente del modelo @biocomsc \n Fuente de los datos #escovid19data', fontsize=12)
    

    
lista_elementos = []    
fig = plt.Figure(figsize=(10, 5), dpi=144)
ax = fig.add_subplot()


anim = FuncAnimation(fig=fig, func=update, init_func=init, frames=len(datos), 
                     interval=150, repeat=False)

#con interval puedes modular la velocidad del video, más alto --> mas lento

In [20]:
from IPython.display import HTML
html = anim.to_html5_video()
HTML(html)

<ipython-input-19-611948b128eb>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-19-611948b128eb>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-19-611948b128eb>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,


In [21]:
anim.save('modelo_biocomsc_badajoz.mp4',dpi=288)

<ipython-input-19-611948b128eb>:90: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 0, 30/rango,
<ipython-input-19-611948b128eb>:94: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango, 30/rango, 100/rango,
<ipython-input-19-611948b128eb>:98: RuntimeWarning: divide by zero encountered in true_divide
  ax.fill_between(rango,100/rango,10,
